In [0]:
!pip install jieba
!git clone https://github.com/Cryptum169/Rake_For_Chinese.git

Cloning into 'Rake_For_Chinese'...
remote: Enumerating objects: 77, done.
remote: Total 77 (delta 0), reused 0 (delta 0), pack-reused 77
Unpacking objects: 100% (77/77), done.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import jieba
import jieba.posseg as pseg
import operator
import json
from collections import Counter


# Data structure for holding data
class Word():
    def __init__(self, char, freq = 0, deg = 0):
        self.freq = freq
        self.deg = deg
        self.char = char

    def returnScore(self):
        return self.deg/self.freq

    def updateOccur(self, phraseLength):
        self.freq += 1
        self.deg += phraseLength

    def getChar(self):
        return self.char

    def updateFreq(self):
        self.freq += 1

    def getFreq(self):
        return self.freq

# Check if contains num
def notNumStr(instr):
    for item in instr:
        if '\u0041' <= item <= '\u005a' or ('\u0061' <= item <='\u007a') or item.isdigit():
            return False
    return True

# Read Target Case if Json
def readSingleTestCases(testFile):
    with open(testFile) as json_data:
        try:
            testData = json.load(json_data)
        except:
            # This try block deals with incorrect json format that has ' instead of "
            data = json_data.read().replace("'",'"')
            try:
                testData = json.loads(data)
                # This try block deals with empty transcript file
            except:
                return ""
    returnString = ""
    for item in testData:
        try:
            returnString += item['text']
        except:
            returnString += item['statement']
    return returnString

def run(rawText):
    # Construct Stopword Lib
    swLibList = [line.rstrip('\n') for line in open("Rake_For_Chinese/data/stoplist/中文停用词表(1208个).txt",'r')]
    # Construct Phrase Deliminator Lib
    conjLibList = [line.rstrip('\n') for line in open("Rake_For_Chinese/data/stoplist/中文分隔词词库.txt",'r')]

    # Cut Text
    rawtextList = pseg.cut(rawText)

    # Construct List of Phrases and Preliminary textList
    textList = []
    listofSingleWord = dict()
    lastWord = ''
    poSPrty = ['m','x','uj','ul','mq','u','v','f']
    meaningfulCount = 0
    checklist = []
    for eachWord, flag in rawtextList:
        checklist.append([eachWord,flag])
        if eachWord in conjLibList or not notNumStr(eachWord) or eachWord in swLibList or flag in poSPrty or eachWord == '\n':
            if lastWord != '|':
                textList.append("|")
                lastWord = "|"
        elif eachWord not in swLibList and eachWord != '\n':
            textList.append(eachWord)
            meaningfulCount += 1
            if eachWord not in listofSingleWord:
                listofSingleWord[eachWord] = Word(eachWord)
            lastWord = ''

    # Construct List of list that has phrases as wrds
    newList = []
    tempList = []
    for everyWord in textList:
        if everyWord != '|':
            tempList.append(everyWord)
        else:
            newList.append(tempList)
            tempList = []

    tempStr = ''
    for everyWord in textList:
        if everyWord != '|':
            tempStr += everyWord + '|'
        else:
            if tempStr[:-1] not in listofSingleWord:
                listofSingleWord[tempStr[:-1]] = Word(tempStr[:-1])
                tempStr = ''

    # Update the entire List
    for everyPhrase in newList:
        res = ''
        for everyWord in everyPhrase:
            listofSingleWord[everyWord].updateOccur(len(everyPhrase))
            res += everyWord + '|'
        phraseKey = res[:-1]
        if phraseKey not in listofSingleWord:
            listofSingleWord[phraseKey] = Word(phraseKey)
        else:
            listofSingleWord[phraseKey].updateFreq()

    # Get score for entire Set
    outputList = dict()
    for everyPhrase in newList:

        if len(everyPhrase) > 5:
            continue
        score = 0
        phraseString = ''
        outStr = ''
        for everyWord in everyPhrase:
            score += listofSingleWord[everyWord].returnScore()
            phraseString += everyWord + '|'
            outStr += everyWord
        phraseKey = phraseString[:-1]
        freq = listofSingleWord[phraseKey].getFreq()
        if freq / meaningfulCount < 0.01 and freq < 3 :
            continue
        outputList[outStr] = score

    sorted_list = sorted(outputList.items(), key = operator.itemgetter(1), reverse = True)
    return sorted_list[:10]

In [0]:
import pandas as pd

df = pd.read_csv("/content/gdrive/My Drive/Dataset/User_profile_id_name.txt",sep = "\t",header= None)
df.columns = columns = ["User_id","name"]
df['User_id'] = df['User_id'].astype('int64')
df=df.sort_values(['User_id']).reset_index(drop=True)
df['tweet'] = ""
df['tweet'][df['name'].isnull()] = 'None'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
flag = 0
u_keywords=""

with open('/content/gdrive/My Drive/Dataset/User_tweets.txt','r',encoding="utf-8") as f:
    for line in f:
        if flag == 0:
            u_index,u_id,u_name = line.split('\t')
            u_id = int(u_id)
            u_str = ""
            flag=1
            continue
        if line == "---------------------------------------------------------------------\n":
            flag=0
            if u_str == 'None' or u_str == "":
              u_keywords = 'None'
            if u_keywords != 'None':
              try:
                keywords_array = []
                for i,j in run(u_str):
                  keywords_array.append(i)
                u_keywords=';'.join(keywords_array)
              except Exception as e:
                print(u_index,":",u_str,":",e)
                u_keywords = 'None'
            df['tweet'][df['User_id'] == u_id] = u_keywords
            u_keywords=""
        u_str +=line.strip('\n') #TODO: remove HTML tags <a href...>

In [0]:
import pandas as pd

User_tweets = pd.read_csv("/content/gdrive/My Drive/Dataset/tweet_keyword.txt",sep = "\t",header= None)
User_tweets.columns = ["drop","User_id","name","tweet"]
User_tweets['User_id'] = User_tweets['User_id'].astype('int64')
del User_tweets['drop']
User_tweets['tweet'][User_tweets['tweet'].isna()] = 'None'

In [0]:
ls = []

def add_to_list(tweet_str):
    #ls.append([i.strip() for i  in tweet_str.split(';')])
    ls.append(tweet_str.split(';'))
    return tweet_str

User_tweets['tweet'].apply(add_to_list)

In [0]:
User_checkin=pd.merge(UserProfile,df,on="User_id")
User_checkin = User_checkin.sort_values(['User_id']).reset_index(drop=True)

for index,row in User_checkin[0:10].iterrows():
  temp = row.tweet.split(';')  + [row.User_id,row.Place]
  print(temp)

['三亚国际电音节', '三亚国际音乐节', '广大乐迷', '儿童手表', '目前', '三亚', '儿童', '雷音', '安全', '视频', 35277, 'Dongcheng District, Beijing']
['组队战胜鬼王', '式神', '人品好', '平安京', '现世活动', '小小黑', '阴阳师手游', '阴阳师', '红包', '鬼王', 48827, 'Shijingshan District, Beijing']
['别样意大利餐', '肉沫豆腐', '关西经典', '亲子盖饭', '特别值得一提的是', '阿当厉害', '红烧鸡腿', '北京主食', '馒头', '寿喜', 101707, 'nan']
['竞园图片产业基地', '投资', '小', '时', '已', '稳定', '局', '德国', '已经', '公司', 103500, 'Chaoyang District, Beijing']
['今天', '明明', '爱情', '人类', '机派', '不幸', '事', '孩子', '味道', '人', 103984, 'Chaoyang District, Beijing']
['None', 104127, 'Chaoyang District, Beijing']
['None', 104174, 'Chaoyang District, Beijing']
['暖暖环游世界', '完美评分', '小米手机', '手机', '最', '小米', '二维码', '试试', '老虎', '刚刚', 104899, 'Beijing Haidian District']
['亚历山大心痛', '反季节旅行', '新西兰疯狂', '传花', '没良心', '倒霉', '朵花', '身', '杜蕾斯', '先是', 120430, 'nan']
['None', 397625, 'Chaoyang District, Beijing']
